In [12]:
from datetime import datetime as dt
import numpy as np
import pandas as pd
import math
import matplotlib.dates as mdates
import os
import matplotlib.pyplot as plt

def load_windata(path):
    import warnings
    warnings.filterwarnings('ignore')
    data = pd.read_csv(path)
    data = data["%time,field.data"]
    data = np.array(data)
    data = data.astype(str)
    data = np.array([l.split(',') for l in data])
    l = np.shape(data)[0]

    newdata = np.zeros((l,5))
    c = 0
    for i in range(0,l):
        try:
            newdata[i,0] = float(data[i][1])
            newdata[i,4] = float(data[i][7])
            newdata[i,3] = float(data[i][5])
            newdata[i,2] = float(data[i][4])
            newdata[i,1] = float(data[i][3])
        except:
            c = c+1
    newdata = pd.DataFrame(newdata, columns = ['Time','U','V','W','Temp'])
    newdata["Time"] = pd.to_datetime(newdata["Time"], unit='s')
    return newdata

In [13]:
w1=pd.read_csv('./Data/Galveston3_Part1_processed.csv')
w2=pd.read_csv('./Data/Galveston3_Part2_processed.csv')

In [14]:
w1a = load_windata('./Data/wind_data_1.csv')
w2a = load_windata('./Data/wind_data_2.csv')
gps1 = pd.read_csv('./Data/gps_1.csv')
gps2 = pd.read_csv('./Data/gps_2.csv')

In [15]:
gps1.head()

,Unnamed: 0,%time,field.header.seq,field.header.stamp,field.header.frame_id,field.tow,field.heading,field.n_sats,field.flags
0,0,1618763951982499143,1,1618763951980361938,NaN,59970600,54594,13,4
1,1,1618763952265709992,2,1618763952265067100,NaN,59970800,54716,13,4
2,2,1618763952470669651,3,1618763952466574907,NaN,59971000,54754,13,4
3,3,1618763952572996338,4,1618763952568240880,NaN,59971200,54766,13,4
4,4,1618763952873061008,5,1618763952872515916,NaN,59971400,54813,13,4


In [16]:
w1a.head()

,Time,U,V,W,Temp
0,2021-04-18 16:39:11.930000128,-4.909,-3.320,-0.229,17.18
1,2021-04-18 16:39:12.029999872,-5.069,-2.824,-0.073,17.06
2,2021-04-18 16:39:12.130000128,-5.291,-2.899,-0.066,17.21
3,2021-04-18 16:39:12.230000128,-5.418,-2.964,0.087,17.28
4,2021-04-18 16:39:12.329999872,-5.608,-3.266,0.365,17.21


In [17]:
def matchtime(wdata,gps):
    import warnings
    warnings.filterwarnings('ignore')
    gtime = gps['%time']
    gtime = pd.to_datetime(gtime, unit='ns')
    angle = gps['field.heading']
    angle = angle/1000
    gps = pd.concat([gtime,angle],axis = 1)
    gps = gps.rename(columns={"%time": "Time", 'field.heading': "Theta"})

    avg_gps = gps.diff()
    avg_gps = avg_gps.reindex(avg_gps.index.drop(0)).reset_index(drop=True)
    avg_gps = avg_gps + gps
    avg_gps=avg_gps.dropna()
    gps = pd.concat([avg_gps,gps])
    # tgps = tgps.reset_index(drop=True)
    gps = gps.sort_values(by=['Time'])
    gps = gps.reset_index(drop=True)
    gps

    threshold = abs(gps['Time'][2]-gps['Time'][0])
    print("Threshold: ",threshold)

    wdata['Theta'] = float(0.00)
    c = 0
    l = wdata.shape[0]
    # print(l)
    for i in range (l):
        diff = abs(gps['Time'][c] - wdata['Time'][i])
        if diff > threshold:
            c = c-1
            diff = abs(gps['Time'][c] - wdata['Time'][i])
        wdata['Theta'][i] = gps['Theta'][c]
        c = c+1
    wdata = wdata[wdata.U != 0]
    return wdata

In [18]:
wdata1 = matchtime(w1a,gps1)
wdata2 = matchtime(w2a,gps2)
df = pd.concat([wdata1,wdata2])
df = df.reset_index(drop=True)

Threshold:  0 days 00:00:00.283210849
Threshold:  0 days 00:00:00.211819688


In [19]:
df.head()

,Time,U,V,W,Temp,Theta
0,2021-04-18 16:39:11.930000128,-4.909,-3.320,-0.229,17.18,54.594
1,2021-04-18 16:39:12.029999872,-5.069,-2.824,-0.073,17.06,54.716
2,2021-04-18 16:39:12.130000128,-5.291,-2.899,-0.066,17.21,54.716
3,2021-04-18 16:39:12.230000128,-5.418,-2.964,0.087,17.28,54.754
4,2021-04-18 16:39:12.329999872,-5.608,-3.266,0.365,17.21,54.754


In [20]:
# ns = -df["U"]*np.sin(df["Theta"]) - df["V"]*np.cos(df["Theta"])
# ew = -df["U"]*np.cos(df["Theta"]) + df["V"]*np.sin(df["Theta"])
# df["Wind_NS"] = ns
# df["Wind_EW"] = ew
# df

In [23]:
ns = df["V"]*np.sin(df["Theta"]) + df["U"]*np.cos(df["Theta"])
ew = df["V"]*np.cos(df["Theta"]) - df["U"]*np.sin(df["Theta"])
df["Wind_NS"] = ns
df["Wind_EW"] = ew
df

,Time,U,V,W,Temp,Theta,Wind_NS,Wind_EW,Wind_Speed,Wind_Theta
0,2021-04-18 16:39:11.930000128,-4.909,-3.320,-0.229,17.18,54.594,4.916860,-3.308348,5.926270,-33.934951
1,2021-04-18 16:39:12.029999872,-5.069,-2.824,-0.073,17.06,54.716,4.040028,-4.165082,5.802563,-45.873174
2,2021-04-18 16:39:12.130000128,-5.291,-2.899,-0.066,17.21,54.716,4.169945,-4.360097,6.033149,-46.277028
3,2021-04-18 16:39:12.230000128,-5.418,-2.964,0.087,17.28,54.754,4.092863,-4.624769,6.175761,-48.491571
4,2021-04-18 16:39:12.329999872,-5.608,-3.266,0.365,17.21,54.754,4.429508,-4.742982,6.489716,-46.957347
...,...,...,...,...,...,...,...,...,...,...
117057,2021-04-18 20:28:34.549999872,-1.899,5.797,-0.913,18.42,174.337,-5.755053,-2.022567,6.100116,19.363615
117058,2021-04-18 20:28:34.660000000,-2.086,5.854,-1.032,18.52,174.337,-5.808040,-2.210744,6.214556,20.838579
117059,2021-04-18 20:28:34.760000000,-2.078,6.136,-1.105,18.57,174.337,-6.090146,-2.208778,6.478316,19.934723
117060,2021-04-18 20:28:34.849999872,-2.135,6.131,-1.115,18.59,174.337,-6.083928,-2.265658,6.492102,20.425353


In [24]:
# df['Wind_Speed'] = (df.U*df.U + df.V*df.V)**(0.5)
# df['Wind_Theta'] = np.degrees(np.arctan(df.U/df.V))
df['Wind_Speed'] = (df.Wind_NS*df.Wind_NS + df.Wind_EW*df.Wind_EW)**(0.5)
df['Wind_Theta'] = np.degrees(np.arctan(df.Wind_EW/df.Wind_NS))
df

,Time,U,V,W,Temp,Theta,Wind_NS,Wind_EW,Wind_Speed,Wind_Theta
0,2021-04-18 16:39:11.930000128,-4.909,-3.320,-0.229,17.18,54.594,4.916860,-3.308348,5.926270,-33.934951
1,2021-04-18 16:39:12.029999872,-5.069,-2.824,-0.073,17.06,54.716,4.040028,-4.165082,5.802563,-45.873174
2,2021-04-18 16:39:12.130000128,-5.291,-2.899,-0.066,17.21,54.716,4.169945,-4.360097,6.033149,-46.277028
3,2021-04-18 16:39:12.230000128,-5.418,-2.964,0.087,17.28,54.754,4.092863,-4.624769,6.175761,-48.491571
4,2021-04-18 16:39:12.329999872,-5.608,-3.266,0.365,17.21,54.754,4.429508,-4.742982,6.489716,-46.957347
...,...,...,...,...,...,...,...,...,...,...
117057,2021-04-18 20:28:34.549999872,-1.899,5.797,-0.913,18.42,174.337,-5.755053,-2.022567,6.100116,19.363615
117058,2021-04-18 20:28:34.660000000,-2.086,5.854,-1.032,18.52,174.337,-5.808040,-2.210744,6.214556,20.838579
117059,2021-04-18 20:28:34.760000000,-2.078,6.136,-1.105,18.57,174.337,-6.090146,-2.208778,6.478316,19.934723
117060,2021-04-18 20:28:34.849999872,-2.135,6.131,-1.115,18.59,174.337,-6.083928,-2.265658,6.492102,20.425353


In [25]:
df.to_csv(r'./Data/wind_data_processed.csv', index = False)
df

,Time,U,V,W,Temp,Theta,Wind_NS,Wind_EW,Wind_Speed,Wind_Theta
0,2021-04-18 16:39:11.930000128,-4.909,-3.320,-0.229,17.18,54.594,4.916860,-3.308348,5.926270,-33.934951
1,2021-04-18 16:39:12.029999872,-5.069,-2.824,-0.073,17.06,54.716,4.040028,-4.165082,5.802563,-45.873174
2,2021-04-18 16:39:12.130000128,-5.291,-2.899,-0.066,17.21,54.716,4.169945,-4.360097,6.033149,-46.277028
3,2021-04-18 16:39:12.230000128,-5.418,-2.964,0.087,17.28,54.754,4.092863,-4.624769,6.175761,-48.491571
4,2021-04-18 16:39:12.329999872,-5.608,-3.266,0.365,17.21,54.754,4.429508,-4.742982,6.489716,-46.957347
...,...,...,...,...,...,...,...,...,...,...
117057,2021-04-18 20:28:34.549999872,-1.899,5.797,-0.913,18.42,174.337,-5.755053,-2.022567,6.100116,19.363615
117058,2021-04-18 20:28:34.660000000,-2.086,5.854,-1.032,18.52,174.337,-5.808040,-2.210744,6.214556,20.838579
117059,2021-04-18 20:28:34.760000000,-2.078,6.136,-1.105,18.57,174.337,-6.090146,-2.208778,6.478316,19.934723
117060,2021-04-18 20:28:34.849999872,-2.135,6.131,-1.115,18.59,174.337,-6.083928,-2.265658,6.492102,20.425353
